In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, models, optimizers, backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.linear_model import LogisticRegression


In [ ]:
df = pd.read_excel('donnees_nettoyees.xlsx')

In [ ]:
print("Fichier chargé !")
print(f"Shape : {df.shape}")
print("\nColonnes présentes :")
print(df.columns.tolist())

Fichier chargé !
Shape : (569, 32)

Colonnes présentes :
['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']


In [ ]:
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis']
# Afficher les dimensions
print("Shape X:", X.shape)
print("Shape y:", y.shape)


Shape X: (569, 30)
Shape y: (569,)


In [ ]:
print(df['diagnosis'].value_counts())

diagnosis
0    357
1    212
Name: count, dtype: int64


In [ ]:
# Découpage 70% train et 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,   # 30% pour le test
    random_state=42, # pour reproductibilité
    stratify=y       # pour garder la proportion des classes
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def evaluate(y_true, y_pred, algo_name=None, epochs=None, return_dict=False):
    acc = accuracy_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    tpr = tp / (tp + fn) if (tp + fn) else 0
    tnr = tn / (tn + fp) if (tn + fp) else 0
    fpr = fp / (tn + fp) if (tn + fp) else 0
    fnr = fn / (tp + fn) if (tp + fn) else 0

    print("\n══════════════════════════════════════════")
    print(f"📌 Model : {algo_name if algo_name else 'Model'}")
    print("══════════════════════════════════════════")
    print(f"Accuracy : {acc*100:.6f}%")
    print(f"TPR (Sensitivity) : {tpr*100:.6f}%")
    print(f"TNR (Specificity) : {tnr*100:.6f}%")
    print(f"FPR : {fpr*100:.6f}%")
    print(f"FNR : {fnr*100:.6f}%")
    print("Confusion matrix:")
    print(f"[[{tn} {fp}]")
    print(f" [{fn} {tp}]]")
    print("══════════════════════════════════════════")

    if return_dict:
        return {
            "Algorithm": algo_name,
            "Accuracy %": round(acc * 100, 6),
            "Data points": len(y_true),
            "Epochs": epochs,
            "FPR %": round(fpr * 100, 6),
            "FNR %": round(fnr * 100, 6),
            "TPR %": round(tpr * 100, 6),
            "TNR %": round(tnr * 100, 6),
        }


1) GRU-SVM (Batch 128, LR=1e-3, Epochs=3000, L2, C=5)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, backend as K

y_train_s = (y_train * 2 - 1).astype(float)
y_test_s  = (y_test * 2 - 1).astype(float)

input_dim = X_train.shape[1]
Xtr = X_train.reshape(-1, 1, input_dim)
Xte = X_test.reshape(-1, 1, input_dim)

def squared_hinge_l2(y_true, y_pred):
    margin = 1 - y_true * y_pred
    return K.mean(K.square(K.maximum(0.0, margin))) + 5 * K.sum(K.square(y_pred))  # C = 5

model = models.Sequential([
    layers.Input(shape=(1, input_dim)),
    layers.GRU(128, return_sequences=False, dropout=0.5),  # Cell size=128, dropout=0.5
    layers.Dense(1, activation='linear')
])

model.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
              loss=squared_hinge_l2)

model.fit(Xtr, y_train_s, epochs=3000, batch_size=128, verbose=0)

y_out = model.predict(Xte).flatten()
y_pred = (np.sign(y_out) > 0).astype(int)

print("\n=== GRU-SVM ===")
evaluate(y_test, y_pred ,algo_name="GRU-SVM ")


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

=== GRU-SVM ===

══════════════════════════════════════════
📌 Model : GRU-SVM 
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 92.187500%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 7.812500%
Confusion matrix:
[[107 0]
 [5 59]]
══════════════════════════════════════════


2) SVM (Batch 128, LR=1e-3, Epochs=3000, L2, C=5)

In [ ]:
#1. Conversion y → -1 / +1 (nécessaire pour la hinge loss)
y_train_svm = np.where(y_train == 0, -1.0, 1.0)
y_test_svm = np.where(y_test == 0, -1.0, 1.0)

#2. Hyperparamètres pour svm
batch_size = 128
learning_rate = 1e-3  # 0.001
epochs = 3000
C = 5

#3. Fonction de perte personnalisée (identique au fichier svm.py du GitHub de l’auteur)
def l2_svm_loss(y_true, y_pred):
    hinge = tf.maximum(1.0 - y_true * y_pred, 0.0)
    squared_hinge = tf.square(hinge)
    return C * tf.reduce_mean(squared_hinge)

#4. Modèle linéaire = SVM linéaire
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(
        1,
        use_bias=True,
        kernel_regularizer=tf.keras.regularizers.l2(1.0 / (2 * C))  # 0.1
    )
])
#5. Compilation avec Adam (exactement comme le papier)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=l2_svm_loss,
    metrics=[]
)

#6. Entraînement avec early stopping
callbacks = [
    EarlyStopping(monitor='loss', patience=300, restore_best_weights=True, verbose=1)
]

history = model.fit(
    X_train, y_train_svm,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
    shuffle=True
)
#7. Prédictions
y_pred_raw = model.predict(X_test, verbose=0)
y_pred_svm = np.sign(y_pred_raw).flatten()
y_pred = np.where(y_pred_svm == -1, 0, 1)

#8. Évaluation
evaluate(
    y_true=y_test,
    y_pred=y_pred,
    algo_name="L2-SVM (notre reproduction fidèle)",
    epochs=3000,
    return_dict=False
)



Epoch 1/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - loss: 21.4649
Epoch 2/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 21.1590 
Epoch 3/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 19.7132
Epoch 4/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 18.8487 
Epoch 5/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 17.5830 
Epoch 6/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 15.7056 
Epoch 7/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 15.7582 
Epoch 8/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 14.1318
Epoch 9/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 13.5687 
Epoch 10/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 12.4343 
Epoch 11/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 12.0206 
Epoch 12/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 11.3825 
Epoch 13/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.5050 
Epoch 14/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 9.7633 
Epoch 15/3000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - los

In [ ]:
base_tree = DecisionTreeClassifier(
    max_depth=2,         # faible profondeur = meilleur généralisation
    min_samples_split=5
)

ada = AdaBoostClassifier(
    estimator=base_tree,
    n_estimators=300,        # nombre de weak learners
    learning_rate=0.5,       # vitesse d'apprentissage
    algorithm='SAMME',     # meilleur pour outputs probabilistes
    random_state=42
)

ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)

evaluate(y_test, ada_pred, algo_name="AdaBoost")

/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(



══════════════════════════════════════════
📌 Model : AdaBoost
══════════════════════════════════════════
Accuracy : 97.660819%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 6.250000%
Confusion matrix:
[[107 0]
 [4 60]]
══════════════════════════════════════════


In [ ]:
# Régression linéaire pour classification binaire
from sklearn.linear_model import LinearRegression

# Entraînement
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Prédiction (on arrondit à 0 ou 1)
y_pred_linreg = (lin_reg.predict(X_test) >= 0.5).astype(int)

# Évaluation
evaluate(y_test, y_pred_linreg, algo_name="Linear Regression")


══════════════════════════════════════════
📌 Model : Linear Regression
══════════════════════════════════════════
Accuracy : 95.321637%
TPR (Sensitivity) : 87.500000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 12.500000%
Confusion matrix:
[[107 0]
 [8 56]]
══════════════════════════════════════════


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300,500],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator__max_depth': [1, 2, 3]
}

grid = GridSearchCV(
    AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=42),
    param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best_model = grid.best_estimator_
ada_pred = best_model.predict(X_test)
evaluate(y_test, ada_pred, algo_name="AdaBoost Tuned")


Best params: {'estimator__max_depth': 2, 'learning_rate': 0.5, 'n_estimators': 200}

══════════════════════════════════════════
📌 Model : AdaBoost Tuned
══════════════════════════════════════════
Accuracy : 97.660819%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 6.250000%
Confusion matrix:
[[107 0]
 [4 60]]
══════════════════════════════════════════


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Entraîner la Régression Logistique
log_reg = LogisticRegression(
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)

log_reg.fit(X_train, y_train)
log_reg_pred = log_reg.predict(X_test)

evaluate(y_test, log_reg_pred, algo_name="Logistic Regression")


══════════════════════════════════════════
📌 Model : Logistic Regression
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════


In [ ]:
# Comparaison des modèles
results = []

# GRU-SVM
results.append(evaluate(y_test, y_pred, algo_name="GRU-SVM", return_dict=True))

# L2-SVM (notre reproduction fidèle)
results.append(evaluate(y_test, y_pred, algo_name="L2-SVM (notre reproduction fidèle)", epochs=3000, return_dict=True))

# Linear Regression
results.append(evaluate(y_test, y_pred_linreg, algo_name="Linear Regression", return_dict=True))

# Créer un DataFrame pour comparaison
df_results = pd.DataFrame(results)
print("\n" + "="*80)
print("📊 COMPARAISON FINALE DES MODÈLES")
print("="*80)
print(df_results.to_string(index=False))

# Meilleur modèle
best_algo = df_results.loc[df_results['Accuracy %'].idxmax()]
print("\n🏆 MEILLEUR MODÈLE :")
print(f"   {best_algo['Algorithm']} avec une précision de {best_algo['Accuracy %']}%")


══════════════════════════════════════════
📌 Model : GRU-SVM
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : L2-SVM (notre reproduction fidèle)
══════════════════════════════════════════
Accuracy : 97.076023%
TPR (Sensitivity) : 93.750000%
TNR (Specificity) : 99.065421%
FPR : 0.934579%
FNR : 6.250000%
Confusion matrix:
[[106 1]
 [4 60]]
══════════════════════════════════════════

══════════════════════════════════════════
📌 Model : Linear Regression
══════════════════════════════════════════
Accuracy : 95.321637%
TPR (Sensitivity) : 87.500000%
TNR (Specificity) : 100.000000%
FPR : 0.000000%
FNR : 12.500000%
Confusion matrix:
[[107 0]
 [8 56]]
══════════════════════════════════════════

📊 COMPARAISON FINALE DES MODÈLES
                         Al